# Retrofitting of a Deep Sea Shipping Fleet

### Problem narrative: fleet retrofit and renewal with stepwise emissions reduction over a fixed time horizon

The achievement of the International Maritime Organization's (IMO) emissions target for the global shipping industry necessitates the participation of a diverse group of stakeholders, including ship owners, cargo owners, fuel suppliers, port authorities, and regulatory bodies. The implementation of this objective would require financing from both the private and public sectors.

This study aims to address the challenge of achieving the IMO's emissions goal by optimizing a vessel fleet's emissions reduction options over a specified period. The problem is framed as a cost-minimization exercise, where the objective is to identify the most cost-effective approach to reduce the fleet's total emissions. The proposed methodology considers a stepwise annual reduction requirement for the fleet's emissions.

### Problem outline

This research examines a diverse fleet of vessels owned and operated by Torvald Klaveness, a Norwegian shipping company. The fleet's total capacity satisfies the yearly transportation needs, and each vessel has a current emission level that can be reduced by implementing retrofits. A retrofit option applies a specific emissions reduction to the relevant vessel. Retrofits entail an investment cost that must be borne in the year they are introduced, as well as a fixed operational cost applied for each subsequent year. Replacing an existing vessel with a new one of the same annual capacity can also be modeled as a retrofit. The total emissions from the fleet must not exceed the annual emissions limit, denoted as $E_y$, and we assume a gradual reduction of this amount.

#### Sets

<table align='left'>
<tr><td>$V$</td><td>The set of vessels in the fleet, indexed by $v$</td></tr>
<tr><td>$R$</td><td>The set of emission retrofit options, indexed by $r$</td></tr>
<tr><td>$T$</td><td>The set of years in the planning horizon, indexed by $t$</td></tr>
</table>

#### Parameters

<table align='left'>
<tr><td>$E^V_v$</td><td>The emissions from vessel $v$ at the start of the planning period</td></tr> 
<tr><td>$\gamma_r$</td><td>The emission reduction (%) effect of retrofit option $r$</td></tr> 
<tr><td>$C^{AI}_{r}$</td><td>The installation cost (USD) of introducing retrofit option $r$</td></tr>
<tr><td>$C^{AO}_{r}$</td><td>The annual operation cost (USD) of introducing retrofit option $r$</td></tr>
<tr><td>$E^{TOT}_t$</td><td>Total allowed fleet emissions for time period $t$</td></tr>
<tr><td>$B^{TOT}_t$</td><td>Total retrofit budget for time period $t$</td></tr>
</table>

#### Variables

<table align='left'>
<tr><td>$x_{vrt}$<td><td>1 if retrofit option $r$ is being installed on vessel $v$ in time period $t$<br>
    0 otherwise<td></tr>
<tr><td>$y_{vrt}$<td><td>1 if retrofit option $r$ is already installed on vessel $v$ in time period $t$<br>
    0 otherwise<td></tr>
</table>

#### Model

<table align='left'>
<tr><td colspan=3>$$ min \sum_{v \in V} \sum_{r \in R}  \sum_{t \in T} (C^{AI}_{r} x_{vrt} + C^{AO}_{r} y_{vrt})$$</td><td>(0)</td></tr>
<tr><td align='left'>subject to</td></tr>
<tr><td>$$\sum_{v\in V} (E^V_v - \sum_{r \in R}\gamma_r E^V_v y_{vrt}) \le E^{TOT}_t,$$</td><td>$$t \in T$$</td><td>(1)</td></tr>
<tr><td>$$\sum_{r \in R}C^{AI}_{r}x_{vrt}\le B^{TOT}_t,$$</td><td>$$v \in V, t \in T$$</td><td>(2)</td></tr> 
<tr><td>$$\sum_{t \in T}x_{vrt}\le 1,$$</td><td>$$v \in V, r \in R$$</td><td>(3)</td></tr> 
<tr><td>$$M \sum_{t \in T}x_{vrt}\ge \sum_{t\in T}y_{vrt},$$</td><td>$$v \in V, r \in R$$</td><td>(4)</td></tr> 
<tr><td>$$y_{vrt} \ge x_{vr(t-1)},$$</td><td>$$v \in V, r \in R, t\in T/\{0\}$$</td><td>(5)</td></tr> 
<tr><td>$$y_{vrt} \ge y_{vr(t-1)},$$</td><td>$$v \in V, r \in R, t\in T/\{0\}$$</td><td>(6)</td></tr> 
<tr><td>$$y_{vrt} - y_{vr(t-1)} = x_{vrt},$$</td><td>$$v \in V, r\in R, t\in T/\{0\}$$</td><td>(7)</td></tr> 
<tr><td>$$x_{vrt} = 0$$</td><td>$$v \in V, r \in R, \text{year today + t} \in \text{new_build_interval}$$</td><td>(8)</td></tr>
<tr><td>$$x_{vrt} = 0$$</td><td>$$v \in V, r \in R, \text{year today + t} \notin \text{docking_interval}$$</td><td>(9)</td></tr>
<tr><td>$$\sum_{v \in V}E^V_v - \sum_{r \in R} \sum_{v \in V}E^V_v \gamma_r y_{vrt} \leq M$$</td><td>$$r \in R, v \in V, t \in T$$</td><td>(10)</td></tr>
    
<tr><td>$$x_{vrt} \in \{0,1\},$$</td><td>$$v\in V, r\in R, t\in T$$</td><td>(11)</td></tr> 
<tr><td>$$y_{vrt} \in \{0,1\},$$</td><td>$$v\in V, r\in R, t\in T$$</td><td>(12)</td></tr> 
</table>

#### Explanation of model

(0) Objective function minimizing investment plus operational cost <br>
(1) The total emissions from the fleet for a particular year must be less than the total emissions allowed<br>
(2) Annual total retrofit cost cannot exceed the annual retrofit budget<br>
(3) A specific emission retrofit option can only be introduced once during the planning horizon for a vessel v<br>
(4) Make sure that a retrofit option is installed if it is used<br>
(5) The retrofit option is put into use the same time period that it is installed<br>
(6) When installed and put into use, the retrofit option will be used for the remaining time periods<br>
(7) Force x to 1 when y switches from 0 to 1<br>
(8) Retrofit option cannot be implemented if vessel's age = 25 (new-build)<br>
(9) A vessel may only be retrofitted in time period t if it is in drydock (its age is its build year plus 5*n)<br>
(10) If a vessel's age is >= 26, reduce its emissions with 25% from that time period t (new-build)<br>
(11, 12) Binary constraints

In [3]:
import os
os.environ['XPRESS'] = '/Users/felix/anaconda3/lib/python3.10/site-packages/xpress/license'
import xpress as xp
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np

In [4]:
# Defining path to excel sheet
path = r'/Users/felix/Library/Mobile Documents/com~apple~CloudDocs/Desktop/Skole/10. Termin/Master/Klaveness/'

# Creating dataframes containing data for the fleet
dfV = pd.read_excel(path + 'KCC_Fleet.xlsx', sheet_name = 'Fleet')
dfR = pd.read_excel(path + 'KCC_Fleet.xlsx', sheet_name = 'Retrofit')
dfT = pd.read_excel(path + 'KCC_Fleet.xlsx', sheet_name = 'Time_horizon')
dfP = pd.read_excel(path + 'KCC_Fleet.xlsx', sheet_name = 'Parameters')

In [5]:
# Calculate the operation cost of every retrofit based on installation cost and operation cost percentage
dfR['Cost_opera'] = [dfR['Cost_install'][i] * dfR['Cost_opera_%'][i] for i in range(len(dfR['Retrofit']))]

# Calculate the annual allowed emissions for the projected time horizon for the fleet based on the last reported annual fleet emissions and annual required emissions reduction
dfT['Allowed_emissions'] = [dfP['Fleet_emissions'][0] * (1 - dfP['Annual_emissions_reduction'][0]) ** (year - dfP['Year_today'][0]) for year in dfT['Year']]

In [6]:
def fleet_model(dfV, dfR, dfT, dfP):
    
    # Defining sets
    # V: vessels
    V = [v for v in range(dfV.shape[0])]
    
    # R: retrofits
    R = [r for r in range(dfR.shape[0])]
    
    # T: time periods
    T = [t for t in range(dfT.shape[0])]
    
    # Big M large enough
    bigM = 100000
    
    # Create optimization problem
    p = xp.problem(name = 'Fleet optimization problem')
    
    # Create and add variables to the optimization problem
    # x: if vessel v is being retrofitted r, this time period t
    x = {(v, r, t): xp.var(vartype = xp.binary, name = 'x_{0}_{1}_{2}'.format(v, r, t))
         for t in T for r in R for v in V}
    
    # y: if vessel v has been retrofitted r, in some time period t
    y = {(v, r, t): xp.var(vartype = xp.binary, name = 'y_{0}_{1}_{2}'.format(v, r, t))
         for t in T for r in R for v in V}
    
    # Add variables
    p.addVariable(x, y)
    
    # Create objective function
    cost_tot = xp.Sum((dfR['Cost_install'][r] * x[v, r, t] + dfR['Cost_opera'][r] * y[v, r, t])
                      / (1 + dfP['r'][0]) ** t for v in V for r in R for t in T)
    p.setObjective(cost_tot, sense = xp.minimize)
        
    # Create constraints
    
    # Constraint 1: Total fleet emissions <= annual allowed fleet emissions
    c1 = [xp.Sum(dfV['Ann_emissions'][v] - xp.Sum(dfR['Reduction'][r] * dfV['Ann_emissions'][v] * y[v, r, t]
            for r in R) for v in V) <= dfT['Allowed_emissions'][t] for t in T]
    
    # Constraint 2: One specific retrofit option may only be used once for each vessel
    c2 = [xp.Sum(x[v, r, t] for t in T) + dfV.iloc[v][dfR['Retrofit'][r]] <= 1 for r in R for v in V]
    
    # Constraint 3: Ensure that x = 1, if at least one y = 1
    c3 = [bigM * (xp.Sum(x[v, r, t] for t in T) + dfV.iloc[v][dfR['Retrofit'][r]]) >= xp.Sum(y[v, r, t] for t in T) for r in R for v in V]
    
    # Constraint 4: Retrofit option installation triggers option use indicator
    c4 = [y[v, r, t] >= x[v, r, t] for t in T for r in R for v in V]
    
    # Constraint 5: Once triggered, option use indicator should last remaining time horizon
    c5 = [y[v, r, t] >= y[v, r, (t - 1)] for t in T[1:] for r in R for v in V]
    
    # Constraint 6: Retrofit option installation trigger option use indicator
    # when y switches from 0 to 1, it trigger x = 1 (x = 0 otherwise)
    c6 = [y[v, r, t] - y[v, r, (t - 1)] == x[v, r, t] for t in T[1:] for r in R for v in V]
    c6 += [y[v, r, 0] - dfV.iloc[v][dfR['Retrofit'][r]] == x[v, r, 0] for r in R for v in V]
    
    # Constraint 7: Retrofit option cannot be implemented if vessel's age = 25 (new-build)
    new_build = 25
    new_build_interval = [[dfV['Build_year'][v] + new_build] for v in V]
    c7 = [x[v, r, t] == 0 for v in V for r in R for t in T if int(dfP['Year_today'][0]) + t in new_build_interval[v] and r != R[-1]]
    
    # Constraint 8: A vessel may only be retrofitted in time period t if its age is its build year plus 5*n
    d = 5 # docking interval
    docking_interval = [[dfP['Year_today'][0] - ((dfP['Year_today'][0] - dfV['Build_year'][v]) % d) + t
                 for t in T if t % d == 0 or t % d == 3] for v in V]
    c8 = [x[v, r, t] == 0 for v in V for r in R for t in T if int(dfP['Year_today'][0]) + t not in docking_interval[v]]
    
    # Constraint 9: If a vessel's age >= 26, reduce its emissions with 25% from that time period t
    # Assumption: a new vessel will keep the already installed retrofit options (new-build from today will be build with some retrofit option) 
    c9 = [x[v, r, t] == 1 for r in [R[-1]] for v in V for t in T if (t + (dfP['Year_today'][0] - dfV['Build_year'][v])) == 25]
    c9 += [dfV.iloc[v][dfR['Retrofit'][r]] == 1 for r in [R[-1]] for v in V if (dfP['Year_today'][0] - dfV['Build_year'][v]) == 25]

    # Add constraints to problem
    p.addConstraint(c1, c2, c3, c4, c5, c6, c7, c8, c9)
    
    # Solve problem
    xp.controls.outputlog = 0
    p.solve()
    
    # Return problem and variables
    return p, x, y

In [ ]:
# Set start time to measure length of code running
start_time = time.time()

# Solve the problem and retrieve the values
p, x, y = fleet_model(dfV, dfR, dfT, dfP)

# Calculate run time
running_time = time.time() - start_time

In [ ]:
# Print run time
if running_time <= 60:
    print("Run time --- %s seconds ---" % round(running_time, 2))
elif running_time <= 3600:
    print("Run time --- %s minutes ---" % round(running_time / 60, 2))
else:
    print("Run time --- %s hours ---" % round(running_time / 3600, 2))

# Print the number of variables, constraints and optimal solution
print('Number of variables:', len(p.getVariable()))
print("Number of constraints:", len(p.getConstraint()))
print('Optimal solution:', round(p.getObjVal(), 1))

In [ ]:
# Function analysing the solution
def solution_analysis(p, x, y, dfV, dfR, dfT):
    
    # Defining sets
    V = [v for v in range(dfV.shape[0])]
    R = [r for r in range(dfR.shape[0])]
    T = [t for t in range(dfT.shape[0])]
    
    # Creating year column
    yr = [t for t in dfT['Year']]
    
    # Calculating cost and emissions
    
    # Annual retrofit fleet investment costs
    CAPEX = [sum([dfR['Cost_install'][r] * p.getSolution(x[v, r, t]) for r in R for v in V])
                  / (1 + dfP['r'][0]) ** t for t in T]
    
    # Annual retrofit fleet operational cost
    OPEX = [sum([dfR['Cost_opera_%'][r] * dfR['Cost_install'][r] * p.getSolution(y[v, r, t]) for r in R for v in V])
                  / (1 + dfP['r'][0]) ** t for t in T]
    
    # Annual voyage fleet cost
    VOYEX = [sum([dfV['Ann_fuel_cons'][v] * dfP['Fuel_cost'][0]
                  - sum([dfV['Ann_fuel_cons'][v] * dfP['Fuel_cost'][0] * dfR['Reduction'][r]
                  * p.getSolution(y[v, r, t]) for r in R]) for v in V])
                  / (1 + dfP['r'][0]) ** t for t in T]
    
    # Annual emission cost
    ENVEX = [sum([dfV['Ann_emissions'][v] * dfP['CO2_tax'][0] - sum([dfV['Ann_emissions'][v] * dfP['CO2_tax'][0]
                  * dfR['Reduction'][r] * p.getSolution(y[v, r, t]) for r in R]) for v in V])
                  / (1 + dfP['r'][0]) ** t for t in T]
    
    # Annual fleet emissions
    ann_emissions = [sum([dfV['Ann_emissions'][v] - sum([dfR['Reduction'][r] * dfV['Ann_emissions'][v]
                      * p.getSolution(y[v, r, t]) for r in R]) for v in V]) for t in T]

    # Creating dataframe with results
    dfRes = pd.DataFrame.from_dict({'Year': T, 'Year_full': yr, 'CAPEX [USD/yr]': CAPEX, 'OPEX [USD/yr]': OPEX,
                                    'VOYEX [USD/yr]': VOYEX, 'ENVEX [USD/yr]': ENVEX, 'Emissions [ton_CO2e/yr]': ann_emissions,
                                    'Allowed emissions [ton_CO2e/yr]': dfT['Allowed_emissions']})
    
    # Annual total cost
    dfRes['Total cost [USD/yr]'] = dfRes['CAPEX [USD/yr]'] + dfRes['OPEX [USD/yr]'] + dfRes['VOYEX [USD/yr]'] + dfRes['ENVEX [USD/yr]']
        
    # Rounding values in dataframe dfRes
    dfRes = dfRes.round({'Emissions': 0, 'Allowed emissions': 0})
    
    # Collecting variable results
    solX = [[[int(p.getSolution(x[v, r, t])) for t in T] for r in R] for v in V]
    solY = [[[int(p.getSolution(y[v, r, t])) for t in T] for r in R] for v in V]
    
    # Creating description column
    dfRes['Vessel'] = [['{}'.format(dfV['Vessel'][v]) for r in R for v in V if solX[v][r][t] == 1] for t in T]
    dfRes['Retrofit'] = [['{}'.format(dfR['Retrofit'][r]) for r in R for v in V if solX[v][r][t] == 1] for t in T]

    # Return dataframe and variable results
    return dfRes, solX, solY

# Gathering results
dfRes, solX, solY = solution_analysis(p, x, y, dfV, dfR, dfT)

In [ ]:
# Adjust dataframe, write it to an excel file and print it
def dataframe_to_excel(dfRes):
    
    # Drop unnecessary column
    dfRes = dfRes.drop('Year', axis = 1)
    
    # Rename column
    dfRes = dfRes.rename({'Year_full': 'Year'}, axis = 1)
    
    # Set Year column as index
    dfRes = dfRes.set_index('Year')
    
    # Write dataframe to an excel file
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        dfRes.to_excel(path + 'Images/Case1/KCC_results_case1.xlsx', sheet_name = 'Results')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        dfRes.to_excel(path + 'Images/Case2/KCC_results_case2.xlsx', sheet_name = 'Results')
    else:
        dfRes.to_excel(path + 'Images/Case3/KCC_results_case3.xlsx', sheet_name = 'Results')
    
    # Return adjusted dataframe
    return dfRes

# Run function
dataframe_to_excel(dfRes)

In [ ]:
print(sum(dfRes['Emissions [ton_CO2e/yr]']))
print(sum(dfRes['Total cost [USD/yr]']))

In [ ]:
# Function plotting results
def results_plot(dfRes):

    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
    ax2 = ax1.twinx()
    
    # Set the width of each bar
    bar_width = 0.4

    # Calculate the center positions for each x value
    x = np.arange(len(dfRes['Year']))

    # Plotting results
    ax1.bar(x - bar_width/2, dfRes['CAPEX [USD/yr]'], color='royalblue', width=bar_width, label='CAPEX')
    ax1.bar(x + bar_width/2, dfRes['OPEX [USD/yr]'], color='seagreen', width=bar_width, label='OPEX')
    
    sns.lineplot(x = 'Year', y = 'Emissions [ton_CO2e/yr]', data = dfRes, ax = ax2, linewidth = 3,
                 color = 'salmon', label = 'Annual emissions', marker = 'o', legend = False)
    sns.lineplot(x = 'Year', y = 'Allowed emissions [ton_CO2e/yr]', data = dfRes, ax = ax2, linewidth = 3,
                 color = 'darkred', label = 'Annual allowed emission', marker = 'o', legend = False)
    
    # Setting title
    plt.title('KCC fleet results', fontsize = 20)
    
    # Creating legend
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    handles = handles1 + handles2
    labels = labels1 + labels2
    fig.legend(handles, labels, bbox_to_anchor=[0.9, 0.88], fontsize = 15)
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    ax2.set_ylabel('ton CO2e', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(range(len(dfRes['Year_full'])))
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_Results_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_Results_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_Results_case3.png')
    
# Run function
results_plot(dfRes)

In [ ]:
# Function plotting results
def eco_results_plot(dfRes):
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
    ax2 = ax1.twinx()
        
    # Plotting results
    sns.barplot(x = 'Year', y = 'CAPEX [USD/yr]', data = dfRes, linewidth = 3, ax = ax1,
                 color = 'salmon', label = 'CAPEX')
    
    sns.lineplot(x = 'Year', y = 'OPEX [USD/yr]', data = dfRes, linewidth = 3, ax = ax1,
                 color = 'blue', label = 'OPEX', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'VOYEX [USD/yr]', data = dfRes, linewidth = 3, ax = ax2,
                 color = 'green', label = 'VOYEX', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'ENVEX [USD/yr]', data = dfRes, linewidth = 3, ax = ax2,
                 color = 'orange', label = 'ENVEX', marker = 'o', legend = False)
    
    # Setting title
    plt.title('KCC fleet retrofit economic results', fontsize = 20)
    
    # Creating legend
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    handles = handles1 + handles2
    labels = labels1 + labels2
    fig.legend(handles, labels, bbox_to_anchor=[0.9, 0.88], fontsize = 15)
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD (CAPEX)', fontsize = 20)
    ax2.set_ylabel('USD (OPEX, VOYEX, ENVEX)', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_Eco_Results_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_Eco_Results_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_Eco_Results_case3.png')
    
# Run function
eco_results_plot(dfRes)

In [ ]:
# Plot the return on investment of retrofit options
def ROI(dfRes):
    
    T = [t for t in range(dfT.shape[0])]
    
    ROI_Flett_Banastar = [((dfV['Ann_fuel_cons'][1] * dfP['Fuel_cost'][0]
                          + dfV['Ann_emissions'][1] * dfP['CO2_tax'][0]) * dfR['Reduction'][0] * t
                          - dfR['Cost_install'][0] * (1 + dfR['Cost_opera_%'][0]))
                          / (1 + dfP['r'][0]) ** t for t in T]
    
    ROI_Air_Baru = [((dfV['Ann_fuel_cons'][8] * dfP['Fuel_cost'][0]
                    + dfV['Ann_emissions'][8] * dfP['CO2_tax'][0]) * dfR['Reduction'][2] * t
                    - dfR['Cost_install'][2] * (1 + dfR['Cost_opera_%'][2]))
                    / (1 + dfP['r'][0]) ** t for t in T]
    
    ROI_Duct_Barracuda = [((dfV['Ann_fuel_cons'][9] * dfP['Fuel_cost'][0]
                          + dfV['Ann_emissions'][9] * dfP['CO2_tax'][0]) * dfR['Reduction'][5] * t
                          - dfR['Cost_install'][5] * (1 + dfR['Cost_opera_%'][5]))
                          / (1 + dfP['r'][0]) ** t for t in T]
    
    ROI_Shaft_Bass = [((dfV['Ann_fuel_cons'][14] * dfP['Fuel_cost'][0]
                      + dfV['Ann_emissions'][14] * dfP['CO2_tax'][0]) * dfR['Reduction'][4] * t
                      - dfR['Cost_install'][4] * (1 + dfR['Cost_opera_%'][4]))
                      / (1 + dfP['r'][0]) ** t for t in T]
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
        
    # Plotting results
    
    sns.lineplot(x = 'Year', y = ROI_Flett_Banastar, data = dfRes, linewidth = 3,
                 color = 'darkorange', label = 'ROI Banastar Flettner rotor', marker = 'o')
    
    sns.lineplot(x = 'Year', y = ROI_Air_Baru, data = dfRes, linewidth = 3,
                 color = 'darkolivegreen', label = 'ROI Baru Air lubrication', marker = 'o')
    
    sns.lineplot(x = 'Year', y = ROI_Duct_Barracuda, data = dfRes, linewidth = 3,
                 color = 'navy', label = 'ROI Barracuda Propeller duct', marker = 'o')
    
    sns.lineplot(x = 'Year', y = ROI_Shaft_Bass, data = dfRes, linewidth = 3,
                 color = 'lemonchiffon', label = 'ROI Bass Shaft generator', marker = 'o')
    
    # Setting title
    plt.title('Return on investments', fontsize = 20)
    
    # Setting labels
    ax1.set_xticks(range(len(dfRes['Year_full'])))
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    plt.savefig(path + 'Images/ROI.png')
    
ROI(dfRes)

In [ ]:
# Plot heatmap for resulting dataframe
def heatmap(dfRes):
    
    # Set the parameter for the size of the plot we want
    plt.rcParams["figure.figsize"] = (20, 10)

    # Dropping unnecessary columns
    dfRes = dfRes.drop(['Year_full', 'Vessel', 'Retrofit'], axis = 1)
    
    # Plotting heatmap for resulting dataframe
    sns.heatmap(dfRes.corr(), cmap = "YlGnBu", annot = True, annot_kws = {"fontsize": 20})
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_results_heatmap_case1.png', bbox_inches = "tight")
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_results_heatmap_case2.png', bbox_inches = "tight")
    else:
        plt.savefig(path + 'Images/Case3/KCC_results_heatmap_case3.png', bbox_inches = "tight")
    
# Run function
heatmap(dfRes)

In [ ]:
# Plot clustermap for resulting dataframe
def clustermap(dfRes):
    
    # Set the parameter for the size of the plot
    plt.rcParams["figure.figsize"] = (20, 10)
    sns.set(font_scale = 1.1)

    # Dropping unnecessary columns
    dfRes = dfRes.drop(['Year_full', 'Vessel', 'Retrofit'], axis = 1)
    
    # Plotting clustermap for resulting dataframe
    sns.clustermap(dfRes.corr(), cmap = "YlGnBu", annot = True, annot_kws={"fontsize": 10},
                   cbar_pos=(.02, .32, .03, .2), linewidths=.75)
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_results_clustermap_case1.png', bbox_inches = "tight")
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_results_clustermap_case2.png', bbox_inches = "tight")
    else:
        plt.savefig(path + 'Images/Case3/KCC_results_clustermap_case3.png', bbox_inches = "tight")
    
# Run function
clustermap(dfRes)

In [ ]:
def lmplot(dfRes):
    
    # Set the parameter for the size of the plot
    plt.rcParams["figure.figsize"] = (10, 10)
    
    # Plotting lmplot for annual emissions and cost
    sns.lmplot(x="Total cost [USD/yr]", y="Emissions [ton_CO2e/yr]", data=dfRes, order=2, height=8, aspect=1.5)

    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_results_lmplot_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_results_lmplot_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_results_lmplot_case3.png')
    
lmplot(dfRes)

In [ ]:
# Function plotting the popularity of each retrofit option
def retrofit_stats(dfRes):
    
    # Create an empty dictionary to store the results
    counts = {}
    
    # Loop through every row in 'Retrofit' column
    for i in dfRes['Retrofit']:
        
        # Loop through every element in every row
        for retrofit in i:
            
            # Add the element to 'counts' and increment the count with 1
            if retrofit in counts:
                counts[retrofit] += 1
            else:
                counts[retrofit] = 1
    
    # Passing initial values
    counts['Air_lubrication'] = 2
    counts['Shaft_generator'] = 2
    
    # Pass the dictionary to a dataframe
    retrofits = pd.DataFrame(counts.items(), columns=['Retrofit', 'Count'])
        
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
    
    # Plotting results
    sns.barplot(x = 'Retrofit', y = 'Count', data = retrofits, linewidth = 1, color = 'salmon')

    # Setting title
    plt.title('Retrofit counts', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticklabels(retrofits['Retrofit'], rotation = 90)
    plt.rc('xtick', labelsize = 15) 
    plt.rc('ytick', labelsize = 15) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_Retrofit_counts_case1.png', bbox_inches = "tight")
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_Retrofit_counts_case2.png', bbox_inches = "tight")
    else:
        plt.savefig(path + 'Images/Case3/KCC_Retrofit_counts_case3.png', bbox_inches = "tight")
    
# Run function
retrofit_stats(dfRes)

In [ ]:
# Function plotting results
def opex_results_plot(dfRes):
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
    
    yr = [t for t in dfT['Year']]
        
    # Plotting results    
    sns.lineplot(x = 'Year', y = 'OPEX [USD/yr]', data = dfRes, linewidth = 3,
                 color = 'blue', label = 'OPEX', marker = 'o', legend = False)
    
    # Setting title
    plt.title('KCC fleet OPEX results', fontsize = 20)
    
    # Creating legend
    plt.legend(prop={'size': 15})
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(range(len(dfRes['Year_full'])))
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_OPEX_Results_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_OPEX_Results_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_OPEX_Results_case3.png')
    
# Run function
opex_results_plot(dfRes)

In [ ]:
# Function plotting results
def voyex_results_plot(dfRes):
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
        
    # Plotting results    
    sns.lineplot(x = 'Year', y = 'VOYEX [USD/yr]', data = dfRes, linewidth = 3,
                 color = 'green', label = 'VOYEX', marker = 'o', legend = False)
    
    # Setting title
    plt.title('KCC fleet VOYEX results', fontsize = 20)
    
    # Creating legend
    plt.legend(prop={'size': 15})
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(range(len(dfRes['Year_full'])))
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_VOYEX_Results_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_VOYEX_Results_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_VOYEX_Results_case3.png')
    
# Run function
voyex_results_plot(dfRes)

In [ ]:
# Function plotting results
def envex_results_plot(dfRes):
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
        
    # Plotting results    
    sns.lineplot(x = 'Year', y = 'ENVEX [USD/yr]', data = dfRes, linewidth = 3,
                 color = 'orange', label = 'ENVEX', marker = 'o', legend = False)
    
    # Setting title
    plt.title('KCC fleet ENVEX results', fontsize = 20)
    
    # Creating legend
    plt.legend(prop={'size': 15})
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(range(len(dfRes['Year_full'])))
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_ENVEX_Results_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_ENVEX_Results_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_ENVEX_Results_case3.png')
    
# Run function
envex_results_plot(dfRes)

In [ ]:
# Function plotting results
def total_results_plot(dfRes):
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
        
    # Plotting results    
    sns.lineplot(x = 'Year', y = 'Total cost [USD/yr]', data = dfRes, linewidth = 3,
                 color = 'yellow', label = 'Total', marker = 'o', legend = False)
    
    # Setting title
    plt.title('KCC fleet total results', fontsize = 20)
    
    # Creating legend
    plt.legend(prop={'size': 15})
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(range(len(dfRes['Year_full'])))
    ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    if dfP['Annual_emissions_reduction'][0] == 0.03:
        plt.savefig(path + 'Images/Case1/KCC_total_Results_case1.png')
    elif dfP['Annual_emissions_reduction'][0] == 0.04:
        plt.savefig(path + 'Images/Case2/KCC_total_Results_case2.png')
    else:
        plt.savefig(path + 'Images/Case3/KCC_total_Results_case3.png')
    
# Run function
total_results_plot(dfRes)

In [ ]:
# Calculating the CII for a vessel

# Defining global lists
Z = [5, 7, 9, 11] # %
CII_score = [0.87, 0.96, 1.06, 1.2]
score_value = ['A', 'B', 'C', 'D', 'E']
years = [2023, 2024, 2025, 2026]

def CII(FC, C, Dt):
    
    # Defining parameters
    x = range(len(years))
    a = 5119
    c = 0.622
    corr = 1e6
    Cfj = 3.17
    
    # Calculating the different CII parameters
    CII_ref = [a * C ** (-c) for i in x]
    CII_att = [(i * Cfj) / (C * Dt) * corr for i in FC]
    CII_req = [CII_ref[i] * (100 - Z[i]) / 100 for i in x]
    CII_res = [CII_att[i] / CII_req[i] for i in x]
    dd_vector = []
    
    # Calculating CII score/dd vector of the vessel
    for i in x:
        if CII_res[i] < CII_score[0]:
            dd_vector.append('A')
        elif CII_score[0] < CII_res[i] < CII_score[1]:
            dd_vector.append('B')
        elif CII_score[1] < CII_res[i] < CII_score[2]:
            dd_vector.append('C')
        elif CII_score[2] < CII_res[i] < CII_score[3]:
            dd_vector.append('D')
        else:
            dd_vector.append('E')
    
    # Passing the CII lists to a dataframe
    dfCII = pd.DataFrame({'Years': years, 'CII_ref': CII_ref, 'CII_att': CII_att,
                          'CII_req': CII_req, 'CII_res': CII_res, 'dd_vector': dd_vector})
    
    # Returning the dataframe
    return dfCII

In [ ]:
def CII_relative():
    
    # Creating CII lines
    CII_relative = {}
    
    # Calculating the CIIs
    for i, score in enumerate(CII_score):
        CII_relative[score_value[i]] = [(100 - z) / 100 * score for z in Z]
        
    # Passing the dictionary to a dataframe
    CII_relative = pd.DataFrame.from_dict(CII_relative)
        
    # Returning dataframe
    return CII_relative

CII_relative = CII_relative()

In [ ]:
def CII_plot(dfV):
        
    # Setting fuel consumption list
    FC_Barcagena = [dfV['Ann_fuel_cons'][0], dfV['Ann_fuel_cons'][0], dfV['Ann_fuel_cons'][0], dfV['Ann_fuel_cons'][0]*0.75]
    FC_Bantry = [dfV['Ann_fuel_cons'][3], dfV['Ann_fuel_cons'][3], dfV['Ann_fuel_cons'][3]*0.81, dfV['Ann_fuel_cons'][3]*0.81]
    FC_Barramundi = [dfV['Ann_fuel_cons'][10], dfV['Ann_fuel_cons'][10]*0.81, dfV['Ann_fuel_cons'][10]*0.81, dfV['Ann_fuel_cons'][10]*0.81]

    # Computing CII results using the CII function
    dfCII_Barcagena = CII(FC_Barcagena, dfV['DWT'][0], dfV['Laden_distance'][0])
    dfCII_Bantry = CII(FC_Bantry, dfV['DWT'][3], dfV['Laden_distance'][3])
    dfCII_Barramundi = CII(FC_Barramundi, dfV['DWT'][10], dfV['Laden_distance'][10])

    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')

    # Plotting results    
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Barcagena, color = 'blue',
                 label = 'CII Barcagena', marker = 'o')
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Bantry, color = 'purple',
                 label = 'CII Bantry', marker = 'o')
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Barramundi, color = 'navy',
                 label = 'CII Barramundi', marker = 'o')

    # Overlaying CII rating vectors
    sns.lineplot(x = years, y = CII_relative['A'], color = 'g')
    plt.text(x = 0.1, y = 0.08, s='CII rating A', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['B'], color = 'lightgreen')
    plt.text(x = 0.1, y = 0.2, s='CII rating B', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['C'], color = 'orange')
    plt.text(x = 0.1, y = 0.35, s='CII rating C', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['D'], color = 'r')
    plt.text(x = 0.1, y = 0.57, s='CII rating D', transform=plt.gca().transAxes, fontsize = 15)
    
    plt.text(x = 0.1, y = 0.82, s='CII rating E', transform=plt.gca().transAxes, fontsize = 15)

    # Adding legend
    plt.legend(fontsize = 15, loc='upper right')

    # Setting title
    plt.title('KCC CII results case 1', fontsize = 20)
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('Relative Carbon Intensity Indicator level', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(dfCII_Barcagena['Years'])
    #ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    plt.savefig(path + 'Images/Case1/KCC_CII_case1.png')

# Running function
CII_plot(dfV)

In [ ]:
def CII_plot(dfV):
        
    # Setting fuel consumption list
    FC_Banastar = [dfV['Ann_fuel_cons'][1], dfV['Ann_fuel_cons'][1]*0.81, dfV['Ann_fuel_cons'][1]*0.81, dfV['Ann_fuel_cons'][1]*0.81*0.75]
    FC_Ballard = [dfV['Ann_fuel_cons'][7], dfV['Ann_fuel_cons'][7], dfV['Ann_fuel_cons'][7]*0.81, dfV['Ann_fuel_cons'][7]*0.81]
    FC_Baffin = [dfV['Ann_fuel_cons'][6], dfV['Ann_fuel_cons'][6]*0.89, dfV['Ann_fuel_cons'][6]*0.89, dfV['Ann_fuel_cons'][6]*0.89]

    # Computing CII results using the CII function
    dfCII_Banastar = CII(FC_Banastar, dfV['DWT'][1], dfV['Laden_distance'][1])
    dfCII_Ballard = CII(FC_Ballard, dfV['DWT'][7], dfV['Laden_distance'][7])
    dfCII_Baffin = CII(FC_Baffin, dfV['DWT'][6], dfV['Laden_distance'][6])

    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')

    # Plotting results    
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Banastar, color = 'blue',
                 label = 'CII Banastar', marker = 'o')
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Ballard, color = 'purple',
                 label = 'CII Ballard', marker = 'o')
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Baffin, color = 'navy',
                 label = 'CII Baffin', marker = 'o')

    # Overlaying CII rating vectors
    sns.lineplot(x = years, y = CII_relative['A'], color = 'g')
    plt.text(x = 0.1, y = 0.25, s='CII rating A', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['B'], color = 'lightgreen')
    plt.text(x = 0.1, y = 0.4, s='CII rating B', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['C'], color = 'orange')
    plt.text(x = 0.1, y = 0.52, s='CII rating C', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['D'], color = 'r')
    plt.text(x = 0.1, y = 0.7, s='CII rating D', transform=plt.gca().transAxes, fontsize = 15)
    
    plt.text(x = 0.1, y = 0.92, s='CII rating E', transform=plt.gca().transAxes, fontsize = 15)

    # Adding legend
    plt.legend(fontsize = 15, loc='upper right')

    # Setting title
    plt.title('KCC CII results case 2', fontsize = 20)
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('Relative Carbon Intensity Indicator level', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(dfCII_Banastar['Years'])
    #ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    plt.savefig(path + 'Images/Case2/KCC_CII_case2.png')

# Running function
CII_plot(dfV)

In [ ]:
def CII_plot(dfV):
        
    # Setting fuel consumption list
    FC_Balzani = [dfV['Ann_fuel_cons'][15], dfV['Ann_fuel_cons'][15], dfV['Ann_fuel_cons'][15], dfV['Ann_fuel_cons'][15]*0.81]
    FC_Balboa = [dfV['Ann_fuel_cons'][5], dfV['Ann_fuel_cons'][5]*0.89, dfV['Ann_fuel_cons'][5]*0.89, dfV['Ann_fuel_cons'][5]*0.81]
    FC_Bakkedal = [dfV['Ann_fuel_cons'][4], dfV['Ann_fuel_cons'][4], dfV['Ann_fuel_cons'][4]*0.81, dfV['Ann_fuel_cons'][4]*0.81]

    # Computing CII results using the CII function
    dfCII_Balzani = CII(FC_Balzani, dfV['DWT'][15], dfV['Laden_distance'][15])
    dfCII_Balboa = CII(FC_Balboa, dfV['DWT'][5], dfV['Laden_distance'][5])
    dfCII_Bakkedal = CII(FC_Bakkedal, dfV['DWT'][4], dfV['Laden_distance'][4])

    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')

    # Plotting results    
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Bakkedal, color = 'blue',
                 label = 'CII Bakkedal', marker = 'o')
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Balboa, color = 'purple',
                 label = 'CII Balboa', marker = 'o')
    sns.lineplot(x = 'Years', y = 'CII_res', data = dfCII_Balzani, color = 'navy',
                 label = 'CII Balzani', marker = 'o')

    # Overlaying CII rating vectors
    sns.lineplot(x = years, y = CII_relative['A'], color = 'g')
    plt.text(x = 0.1, y = 0.08, s='CII rating A', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['B'], color = 'lightgreen')
    plt.text(x = 0.1, y = 0.21, s='CII rating B', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['C'], color = 'orange')
    plt.text(x = 0.1, y = 0.33, s='CII rating C', transform=plt.gca().transAxes, fontsize = 15)

    sns.lineplot(x = years, y = CII_relative['D'], color = 'r')
    plt.text(x = 0.1, y = 0.58, s='CII rating D', transform=plt.gca().transAxes, fontsize = 15)
    
    plt.text(x = 0.1, y = 0.81, s='CII rating E', transform=plt.gca().transAxes, fontsize = 15)

    # Adding legend
    plt.legend(fontsize = 15, loc='upper right')

    # Setting title
    plt.title('KCC CII results case 3', fontsize = 20)
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('Relative Carbon Intensity Indicator level', fontsize = 20)
    
    # Setting axis ticks
    ax1.set_xticks(dfCII_Bakkedal['Years'])
    #ax1.set_xticklabels(dfRes['Year_full'], rotation = 90)
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    plt.savefig(path + 'Images/Case3/KCC_CII_case3.png')

# Running function
CII_plot(dfV)

In [ ]:
dfP.T

In [ ]:
dfR

In [ ]:
dfV

In [ ]:
dfT

In [ ]:
# Comarison of the results for the three cases

def comparison():
    
    df1 = pd.read_excel(path + '/Images/Case1/KCC_results_case1.xlsx')
    df2 = pd.read_excel(path + '/Images/Case2/KCC_results_case2.xlsx')
    df3 = pd.read_excel(path + '/Images/Case3/KCC_results_case3.xlsx')
    
    # Creating figure
    fig, ax1 = plt.subplots(figsize = (20, 10))
    sns.set_style(style = 'dark')
    ax2 = ax1.twinx()
        
    # Plotting results    
    sns.lineplot(x = 'Year', y = 'Allowed emissions [ton_CO2e/yr]', data = df1, linewidth = 3, ax = ax2,
                 color = 'darkgreen', label = 'Case 1 emissions', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'Allowed emissions [ton_CO2e/yr]', data = df2, linewidth = 3, ax = ax2,
                 color = 'navy', label = 'Case 2 emissions', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'Allowed emissions [ton_CO2e/yr]', data = df3, linewidth = 3, ax = ax2,
                 color = 'darkorange', label = 'Case 3 emissions', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'Total cost [USD/yr]', data = df1, linewidth = 3, ax = ax1,
                 color = 'seagreen', label = 'Case 1 total cost', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'Total cost [USD/yr]', data = df2, linewidth = 3, ax = ax1,
                 color = 'blue', label = 'Case 2 total cost', marker = 'o', legend = False)
    
    sns.lineplot(x = 'Year', y = 'Total cost [USD/yr]', data = df3, linewidth = 3, ax = ax1,
                 color = 'orange', label = 'Case 3 total cost', marker = 'o', legend = False)
    
    # Setting title
    plt.title('Case 1, 2 & 3 comparison for emissions and total costs', fontsize = 20)
    
     # Creating legend
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    handles = handles1 + handles2
    labels = labels1 + labels2
    fig.legend(handles, labels, bbox_to_anchor=[0.9, 0.88], fontsize = 15)
    
    # Setting labels
    ax1.set_xlabel('Year', fontsize = 20)
    ax1.set_ylabel('USD', fontsize = 20)
    ax2.set_ylabel('ton CO2e', fontsize = 20)
    
    # Setting axis ticks
    plt.rc('xtick', labelsize = 20) 
    plt.rc('ytick', labelsize = 20) 
    
    # Setting grid
    plt.grid(linestyle = '-.')
    
    # Saving figure
    plt.savefig(path + 'Images/Comparison.png')
    
comparison()